In [ ]:
import requests
import pandas as pd
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def fetch_ipo_fudao_data(total_pages_to_scrape=103):
    # 接口地址
    url = "https://datacenter-web.eastmoney.com/api/data/v1/get"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Referer': 'https://data.eastmoney.com/xg/ipo/fd.html',
        'Accept': 'application/json, text/javascript, */*; q=0.01'
    }

    # 配置自动重试机制 (防止网络波动导致的中断)
    session = requests.Session()
    retry_strategy = Retry(total=3, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("https://", adapter)
    session.mount("http://", adapter)

    all_data_list = []
    
    # ----------------------------------------------------
    # 根据你的数据反馈，使用正确的参数配置
    # ----------------------------------------------------
    params_base = {
        'sortColumns': 'RECORD_DATE,ORG_CODE', # 根据数据里的日期字段猜测排序
        'sortTypes': '-1,-1',
        'pageSize': '50',
        # 注意：这里假设你用的 reportName 能返回你提供的那段 JSON
        # 如果代码跑不通，请检查 reportName 是否为 'RPT_IPO_FUDAO' 或 'RPT_Public_TutorInfo'
        'reportName': 'RPT_IPO_TUTRECORD', 
        'columns': 'ALL',
        'source': 'WEB',
        'client': 'WEB'
    }

    for page in range(1, total_pages_to_scrape + 1):
        print(f"[{page}/{total_pages_to_scrape}] 正在爬取...", end="")
        
        params = params_base.copy()
        params['pageNumber'] = page

        try:
            # timeout 设置为 20 秒
            response = session.get(url, headers=headers, params=params, timeout=20)
            
            if response.status_code == 200:
                data_json = response.json()
                
                if data_json.get('result') and data_json['result'].get('data'):
                    items = data_json['result']['data']
                    print(f" 成功获取 {len(items)} 条")
                    
                    # -------------------------------------------------------
                    # 【核心修改】基于你提供的 JSON 数据进行字段映射
                    # -------------------------------------------------------
                    for item in items:
                        row = {
                            '辅导对象': item.get('TUTOR_OBJECT'),      # 原数据: TUTOR_OBJECT
                            '辅导机构': item.get('TUTOR_ORG'),         # 原数据: TUTOR_ORG
                            '备案时间': item.get('RECORD_DATE'),       # 原数据: RECORD_DATE
                            '辅导状态': item.get('TUTOR_PROCESS_STATE'), # 原数据: TUTOR_PROCESS_STATE
                            '派出机构': item.get('DISPATCH_ORG'),      # 原数据: DISPATCH_ORG (可作为地区参考)
                            '报告类型': item.get('REPORT_TYPE'),       # 原数据: REPORT_TYPE
                            '报告标题': item.get('REPORT_TITLE')       # 原数据: REPORT_TITLE
                        }
                        all_data_list.append(row)
                else:
                    print("\n数据已爬完（接口无返回）。")
                    break
            else:
                print(f" 接口报错: {response.status_code}")
                
            time.sleep(1) # 每一页暂停1秒，防止封IP

        except Exception as e:
            print(f"\n第 {page} 页失败: {e}")
            continue

    # 保存数据
    if all_data_list:
        df = pd.DataFrame(all_data_list)
        print(f"\n>>> 爬取结束，共获取 {len(df)} 条数据 <<<")
        print(df.head())
        
        filename = "ipo_fudao_data_fixed.xlsx"
        df.to_excel(filename, index=False)
        print(f"文件已保存为: {filename}")
    else:
        print("未抓取到任何数据。")

if __name__ == "__main__":
    fetch_ipo_fudao_data()

[1/103] 正在爬取...
数据已爬完（接口无返回）。
未抓取到任何数据。
